In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
from shapely.geometry import box

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import colors, patheffects
import cartopy.io.img_tiles as cimgt
import cartopy.crs as ccrs
proj = ccrs.PlateCarree()

In [ ]:
import hydromt
from  hydromt.log import setuplog
from hydromt.workflows.basin_mask import get_basin_geometry
logger = setuplog("basins", log_level=10)

In [ ]:
# read data from artifacts using the data catalog
data_catalog = hydromt.DataCatalog()
ds = data_catalog.get_rasterdataset("hydro_merit")
gdf_bas_index = data_catalog.get_geodataframe("hydro_merit_index")

In [ ]:
flwdir = hydromt.flw.flwdir_from_da(ds['flwdir'], ftype='d8')
feats = flwdir.streams(mask=ds['strord']>=7)
gdf_riv = gpd.GeoDataFrame.from_features(feats).set_index("idxs")

In [ ]:
# get the basin boundary based on a point location
# {'basin': [x, y]}

xy=[12.6, 45.8]
gdf_xy = gpd.GeoDataFrame(geometry=gpd.points_from_xy(x=[xy[0]], y=[xy[1]]), crs=4326)
gdf_bas, _ = get_basin_geometry(
    ds,
    kind='basin',
    xy=xy,
    gdf_bas=gdf_bas_index,
    logger=logger,
)

In [ ]:
# plot results
extent = np.array(gdf_bas.buffer(0.1).total_bounds)[[0, 2, 1, 3]]
fig = plt.figure(figsize=(6,8))
ax = plt.subplot(projection=proj)
ax.set_extent(extent, crs=proj)
ax.add_image(cimgt.QuadtreeTiles(), 12)
gdf_bas.boundary.plot(ax=ax, edgecolor='k')
gdf_riv.plot(ax=ax, color='blue', alpha=0.7)
gdf_xy.plot(ax=ax, markersize=40, c='red')

In [ ]:
# {'subbasin': [x, y], 'strord':7, 'bounds': [12.2, 45.7, 12.9, 46.45]}
# when an initial bounds is provided the basin index is not used
bounds = [12.1, 45.5, 12.9, 46.5]
gdf_bounds = gpd.GeoDataFrame(geometry=[box(*bounds)], crs=4326)
gdf_bas, gdf_out = get_basin_geometry(
    ds,
    kind='subbasin',
    xy=xy,
    strord=7,
    bounds=bounds,
    logger=logger,
)

In [ ]:
# plot results
extent = gdf_bounds.buffer(0.05).total_bounds[[0, 2, 1, 3]]
fig = plt.figure(figsize=(6,8))
ax = plt.subplot(projection=proj)
ax.set_extent(extent, crs=proj)
ax.add_image(cimgt.QuadtreeTiles(), 12)
gdf_bas.boundary.plot(ax=ax, edgecolor='k')
gdf_riv.plot(ax=ax, color='blue', alpha=0.7)
gdf_out.plot(ax=ax, markersize=40, c='red', zorder=2)
gdf_bounds.boundary.plot(ax=ax, edgecolor='k', ls='--')

In [ ]:
# {'subbasin': [xmin, ymin, xmax, ymax], 'strord':threshold, 'bounds': [xmin, ymin, xmax, ymax]}
# the subbasin outlfow point is based on the bbox
bbox = [12.50, 45.72, 12.7, 46]
gdf_bbox = gpd.GeoDataFrame(geometry=[box(*bbox)], crs=4326)
gdf_bas, gdf_out = get_basin_geometry(
    ds,
    kind='subbasin',
    bbox=bbox,
    strord=8,
    bounds=bounds,
    logger=logger,
)

In [ ]:
# plot results
extent = gdf_bounds.buffer(0.05).total_bounds[[0, 2, 1, 3]]
fig = plt.figure(figsize=(6,8))
ax = plt.subplot(projection=proj)
ax.set_extent(extent, crs=proj)
ax.add_image(cimgt.QuadtreeTiles(), 12)
gdf_bas.boundary.plot(ax=ax, edgecolor='k')
gdf_riv.plot(ax=ax, color='blue', alpha=0.7)
gdf_out.plot(ax=ax, markersize=40, c='red', zorder=2)
gdf_bbox.boundary.plot(ax=ax, edgecolor='r', ls='-')
gdf_bounds.boundary.plot(ax=ax, edgecolor='k', ls='--')

In [ ]:
# {'interbasin': [xmin, ymin, xmax, ymax], 'strord':threshold}
gdf_bas, gdf_out = get_basin_geometry(
    ds,
    kind='interbasin',
    bbox=bbox,
    strord=8,
    buffer=20,
    logger=logger,
)

In [ ]:
# plot results
extent = gdf_bounds.buffer(0.05).buffer(0.02).total_bounds[[0, 2, 1, 3]]
fig = plt.figure(figsize=(6,8))
ax = plt.subplot(projection=proj)
ax.set_extent(extent, crs=proj)
ax.add_image(cimgt.QuadtreeTiles(), 12)
gdf_bas.boundary.plot(ax=ax, edgecolor='k', zorder=2)
gdf_riv.plot(ax=ax, color='blue', alpha=0.7)
gdf_out.plot(ax=ax, markersize=40, c='red', zorder=2)
gdf_bbox.boundary.plot(ax=ax, edgecolor='r', ls='-', zorder=1)
gdf_bounds.boundary.plot(ax=ax, edgecolor='k', ls='--', zorder=1)